In [39]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
sales_df = pd.read_csv('out/sales_aggregated.csv', index_col=0)

In [41]:
sales_df = sales_df.transpose()

In [42]:
sales_df.columns.name = None
sales_df = sales_df.rename_axis("Date")
sales_df = sales_df.reset_index()

sales_df.head()

,Date,SCHAB Z KOŚCIĄ,SCHAB BEZ KOŚCI,KARCZEK BEZ KOŚCI,SZYNKA Z KOŚCIA,SZYNKA BEZ KOŚCI,ŁOPATKA Z KOŚCIA,ŁOPATKA BEZ KOŚCI,ŁOPATKA MIELONA,ŻEBERKO,...,SALCESON,KASZANKA,PASZTETOWA,BOCZEK WEDZ-GOT,BOCZEK WĘDZONY,PODGZRDLE WĘDZONE,GOLONKA WĘDZONA,ŻEBERKA WĘDZONE,PARÓWKI,WĘDLINY
0,01_2015,181.920,192.732,156.977,0.00,177.450,280.98,157.570,244.28,250.190,...,42.47,97.598,34.470,3.490,87.550,33.40,0.72,2.24,0.00,29.580
1,02_2015,174.210,179.310,171.419,2.84,213.170,246.35,165.918,211.50,263.570,...,41.65,66.740,20.947,16.515,56.850,18.72,0.96,0.00,0.00,27.040
2,03_2015,391.492,331.325,301.939,62.11,385.080,378.56,377.568,392.25,435.851,...,64.70,107.050,33.805,49.825,82.562,27.23,2.14,0.91,31.61,34.290
3,04_2015,330.390,177.610,228.790,26.91,219.120,450.51,218.150,361.44,300.840,...,51.75,79.780,33.255,44.860,58.500,75.57,0.00,0.00,37.69,28.973
4,05_2015,459.240,159.792,272.260,33.65,215.273,279.39,257.570,363.41,387.389,...,49.72,103.800,51.710,53.247,77.560,39.19,0.92,3.48,14.22,43.280


### Melting the data to a better format

In [43]:
sales_df = sales_df.melt(id_vars=["Date"], var_name='Product', value_name='Sales in kg')

In [44]:
sales_df

,Date,Product,Sales in kg
0,01_2015,SCHAB Z KOŚCIĄ,181.920
1,02_2015,SCHAB Z KOŚCIĄ,174.210
2,03_2015,SCHAB Z KOŚCIĄ,391.492
3,04_2015,SCHAB Z KOŚCIĄ,330.390
4,05_2015,SCHAB Z KOŚCIĄ,459.240
...,...,...,...
5395,08_2023,WĘDLINY,513.860
5396,09_2023,WĘDLINY,580.460
5397,10_2023,WĘDLINY,691.890
5398,11_2023,WĘDLINY,622.950


### Feature engineering, encoding

In [45]:
def extract_season(value):
    month = str(value).split('_')[0]
    if month in ['12', '01', '02']: return "WIN"
    elif month in ['03', '04', '05']: return "SPR"
    elif month in ['06', '07', '08']: return "SUM"
    elif month in ['09', '10', '11']: return "FAL"

def include_holidays(value):
    month = str(value).split('_')[0]
    if month in ['03', '04']: return "EASTER"
    if month == "12": return "CHRISTMAS"


def feature_engineering(df):
    df['Holiday'] = df['Date'].apply(include_holidays)
    df['Season'] = df['Date'].apply(extract_season)
    return df
    
    
def encoding(df):
    return pd.get_dummies(df, columns=['Holiday', 'Season'], dtype=int)


    
    

sales_df = feature_engineering(sales_df.copy())
sales_df = encoding(sales_df.copy())

sales_df

,Date,Product,Sales in kg,Holiday_CHRISTMAS,Holiday_EASTER,Season_FAL,Season_SPR,Season_SUM,Season_WIN
0,01_2015,SCHAB Z KOŚCIĄ,181.920,0,0,0,0,0,1
1,02_2015,SCHAB Z KOŚCIĄ,174.210,0,0,0,0,0,1
2,03_2015,SCHAB Z KOŚCIĄ,391.492,0,1,0,1,0,0
3,04_2015,SCHAB Z KOŚCIĄ,330.390,0,1,0,1,0,0
4,05_2015,SCHAB Z KOŚCIĄ,459.240,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
5395,08_2023,WĘDLINY,513.860,0,0,0,0,1,0
5396,09_2023,WĘDLINY,580.460,0,0,1,0,0,0
5397,10_2023,WĘDLINY,691.890,0,0,1,0,0,0
5398,11_2023,WĘDLINY,622.950,0,0,1,0,0,0


### Transforming the data to fit the model

In [46]:
# converting Data clumn to pandas datetime and setting it as the index
sales_df['Date'] = pd.to_datetime(sales_df['Date'], format='%m_%Y')
sales_df.set_index('Date', inplace=True)

In [47]:
#extracting a single product sales
schab_z_koscia_sales = sales_df[sales_df['Product'] == 'SCHAB Z KOŚCIĄ']

schab_z_koscia_sales

,Product,Sales in kg,Holiday_CHRISTMAS,Holiday_EASTER,Season_FAL,Season_SPR,Season_SUM,Season_WIN
Date,,,,,,,,
2015-01-01,SCHAB Z KOŚCIĄ,181.920,0,0,0,0,0,1
2015-02-01,SCHAB Z KOŚCIĄ,174.210,0,0,0,0,0,1
2015-03-01,SCHAB Z KOŚCIĄ,391.492,0,1,0,1,0,0
2015-04-01,SCHAB Z KOŚCIĄ,330.390,0,1,0,1,0,0
2015-05-01,SCHAB Z KOŚCIĄ,459.240,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
2023-08-01,SCHAB Z KOŚCIĄ,206.840,0,0,0,0,1,0
2023-09-01,SCHAB Z KOŚCIĄ,214.360,0,0,1,0,0,0
2023-10-01,SCHAB Z KOŚCIĄ,236.590,0,0,1,0,0,0


### Splitting the data

### Testing the model
#### TO DO:
- research about ARIMA and SARIMA
- engineer more features


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX


c:\Users\damia\anaconda3\envs\DataModelingAndAnalitycs\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


                                      SARIMAX Results                                      
Dep. Variable:                         Sales in kg   No. Observations:                  108
Model:             SARIMAX(1, 1, 1)x(1, 1, [], 12)   Log Likelihood                -569.180
Date:                             Sun, 17 Nov 2024   AIC                           1158.360
Time:                                     21:55:06   BIC                           1183.899
Sample:                                 01-01-2015   HQIC                          1168.679
                                      - 12-01-2023                                         
Covariance Type:                               opg                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Holiday_CHRISTMAS     0.0181   4.12e+04   4.39e-07      1.000   -8.08e+04    8.08e+04
Holida